In [1]:
import os
import emcee
import reddemcee

import numpy as np

from glob import glob
from scipy import stats
from pyazr import azure2
from multiprocess import Pool, current_process

# Restrict processes to one thread only
os.environ['OMP_NUM_THREADS'] = '1'

# Define the data labels (in AZURE2 order)
labels = ["Meyer et al. (1976) - 84.3 deg",
          "Meyer et al. (1976) - 114.5 deg",
          "Meyer et al. (1976) - 144.1 deg", 
          "LUNA HPGe (2023)", 
          "LUNA BGO (2023)",
          "Felsenkeller (2023)",
          "ATOMKI (2023)",
          "Notre Dame (2023) - 0 deg",
          "Notre Dame (2023) - 55 deg",
          "Burtebaev et al. (2008)",
          "Lamb et al. (1957)",
          "Bailey et al. (1950)",
          "Vogl et al. (1963)",
          "Rolfs et al. (1974) - 0 deg",
          "Rolfs et al. (1974) - 90 deg"]

# Define the parameters prior distributions
priors = [
    stats.norm(1.63,0.12),

    stats.uniform(2.365, 0.020), stats.uniform(2e4, 3e4), stats.uniform(-1, 1),
    stats.uniform(-1e4, 4e4),

    stats.uniform(3.500, 0.020), stats.uniform(4e4, 3e4), stats.uniform(-1, 1), stats.uniform(-0.1, 0.2),
    stats.uniform(-1e4, 2e4), stats.uniform(-2e4, 2e5),

    stats.uniform(3.540, 0.020), stats.uniform(3e4, 3e4),
    
    stats.lognorm(0.05),
    stats.lognorm(0.05),
    stats.lognorm(0.05),
    stats.lognorm(0.069),
    stats.lognorm(0.079),
    stats.lognorm(0.10),
    stats.lognorm(0.06),
    stats.lognorm(0.10),
    stats.lognorm(0.10),
    stats.lognorm(0.10),
    stats.uniform(0.5, 2),
    stats.uniform(0.5, 2),
    stats.uniform(0.5, 2),
    stats.uniform(0.5, 2),
    stats.uniform(0.5, 2)
]

# Minimization variables
nsteps   = 10000        # How many steps should each walker take?
nprocs   = 20           # How many Python processes do you want to allocate?
ntemps   = 10           # How many tempearature levels do you want to use?
ndim     = len(priors)  # How many parameters are you fitting?
nwalkers = 2 * ndim     # How many walkers do you want to use?

In [3]:
# We read the .azr file and set the external capture file to speed up the calculation
azr = azure2('12c_pg.azr', nprocs=nprocs)

# We get the initial values from AZURE2
theta0 = azr.params
ntheta = len(theta0)

# We'll read the data from the output file since it's already in the center-of-mass frame
y = azr.cross
yerr = azr.cross_err

In [4]:
# Prior log probability
def lnp( theta ):
    return np.sum([pi.logpdf(t) for (pi, t) in zip(priors, theta)])

# Log likelihood
def lnl( theta ):
    res = 0
    try: proc = int(current_process().name.split('-')[1]) - 1 # We want to get the numbe r of the process to call the right AZURE2 port
    except: proc = 0   
    mu = azr.calculate( theta[:ntheta], proc=proc )
    for i in range( len( mu ) ):
        idx = ntheta + i
        res += np.sum( - 0.5 * np.log(2 * np.pi * pow(yerr[i], 2) ) - 0.5 * pow((mu[i] - y[i] * theta[idx]) / (yerr[i] * theta[idx]), 2) )
    return res

In [5]:
# Prepare initial walker positions from the prior
p0 = np.zeros( (ntemps,nwalkers,ndim) )
for t in range(ntemps):
    for i in range(nwalkers):
        for j in range(ndim):
            p0[t,i,j] = priors[j].rvs()

In [ ]:
# Prepare the files to write the chain from each temperature
backends = []
for t in range(ntemps):
    backends.append( emcee.backends.HDFBackend('results/ptemcee/samples_{}.h5'.format(t)) )
    backends[t].reset(nwalkers, ndim)

# Start the sampling
with Pool( nprocs ) as pool:
    sampler = reddemcee.PTSampler( nwalkers, ndim, lnl, lnp, ntemps=ntemps, pool=pool, backend=backends )
    sampler.run_mcmc(list(p0), nsteps)